# Proefening 1

Welkom bij de Geonovum workshop! Dit is de eerste oefening.
Als een reminder: Zorg dat je de volgende dingen hebt voorbereid:
- Je hebt git geinstalleerd en de repository gecloned
- Je hebt het install.sh script gerunned, en de conda environment geonovum_pointcloud geactiveerd in dit notebook
- je hebt Cloudcompare, Qgis, of een andere tool om pointclouds te visualiseren geinstalleerd

Zo ja, dan kunnen we aan de slag!

### Package imports

In [13]:
import laspy as lp
import numpy as np
import open3d as o3d
import os
import yaml
import logging
from typing import Optional
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [14]:
### Load environment variables
def load_yaml(path_to_yaml: str):
    """
    Load yaml file
    """
    with open(path_to_yaml, "r") as file:
        return yaml.safe_load(file)


env = load_yaml("env.yml")
data_folder = env["data_folder"]
output_folder = Path(data_folder) / "output"
bbox = env["bbox"]
bgt_name = env["bgt_name"]
subset = env["subset"]

### Handige functies

Onderstaande functies helpen ons in deze workshop. Bij elke functie zie je een korte beschrijving van wat hij doet.

In [15]:
def get_files_in_directory(directory: str):
    """
    Get all files with a specific extension in a directory.
    """

    files = [
        os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".las")
    ]
    return files


def lasdata_loader(filepaths: str | list, single_array: bool = True):
    """
    Load one or multiple .las or .laz files.
    Can return the data as a single numpy array (if single_array=True)
    or as a list of numpy arrays (if single_array=False).
    """
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    all_points = []
    for filepath in filepaths:
        las = lp.read(filepath)
        coords = np.vstack((las.points.x, las.points.y, las.points.z))
        point_cloud = coords.transpose()
        all_points.append(point_cloud)

    if single_array:
        all_points = np.concatenate(all_points)

    return all_points


def to_las(
    points: np.ndarray, colors: Optional[np.ndarray], extra_dim: Optional[dict]
) -> lp.LasData | None:
    """
    Converts data to laspy format.
    Laspy RGB channels must be of 16-bit format 0-65535.
    These default for the Pointcloud class is 0-255.
    """
    header = lp.LasHeader(point_format=3, version="1.2")

    las = lp.LasData(header)
    las.x = points[:, 0]
    las.y = points[:, 1]
    las.z = points[:, 2]

    if colors is not None:
        point_colors = colors.astype(np.uint16)
        if points.shape[0] != point_colors.shape[0]:
            logging.info("Points and color length are not equal.")
            return
        las.red = point_colors[:, 0]
        las.green = point_colors[:, 1]
        las.blue = point_colors[:, 2]

    if extra_dim is not None:
        if not isinstance(extra_dim, dict):
            logging.info(
                "extra_dim should be a dictionary with the dimname as key, and the value a N1 array or list of values"
            )
            return

        for key, value in extra_dim.items():
            if points.shape[0] != value.shape[0]:
                logging.info("Points and extra dimension length are not equal.")
                return

            if key != "classification":
                las.add_extra_dim(lp.ExtraBytesParams(name=key, type="uint32"))
                setattr(las, key, value)
            else:
                las.classification = value

    return las


### Kleuren voor open3D
def generate_rgb_colors(n_colors):
    """
    Genereer een set van N verschillende RGB kleuren
    Returns: numpy array van shape (n_colors, 3) met RGB waarden tussen 0-255
    """
    # Gebruik een colormap om goed onderscheidbare kleuren te krijgen
    colormap = plt.colormaps["tab10"]  # 'tab10', 'Set3', 'viridis', etc.

    colors = []
    for i in range(n_colors):
        # Krijg kleur van colormap (waarden tussen 0-1)
        rgba = colormap(i / max(1, n_colors - 1))
        # Converteer naar RGB (0-255)
        rgb = [int(rgba[j] * 255) for j in range(3)]
        colors.append(rgb)

    return np.array(colors)


# Pas kleuren toe op punten gebaseerd op hun grid classificatie
def apply_colors_to_points(points_with_grid, color_palette):
    """
    Pas RGB kleuren toe op punten gebaseerd op hun grid classificatie

    Args:
        points_with_grid: numpy array met shape (n_points, 4) - X,Y,Z,grid_class
        color_palette: numpy array met shape (n_colors, 3) - RGB kleuren

    Returns:
        numpy array met shape (n_points, 3) - RGB kleuren voor elk punt
    """
    point_colors = np.zeros((len(points_with_grid), 3))

    for i, point in enumerate(points_with_grid):
        grid_class = int(point[3])  # Grid classificatie uit 4e kolom
        point_colors[i] = color_palette[grid_class]

    return point_colors.astype(np.uint8)

### Stap 1: De las data laden

In [16]:
data_folder = "data/utrecht_subset"  # Pad naar de folder met de las/laz files
output_folder = Path(data_folder) / "output"
os.makedirs(output_folder, exist_ok=True)

In [17]:
lasfiles = get_files_in_directory(data_folder)
print(lasfiles)

lasfile_path = [
    filepath
    for filepath in lasfiles
    if filepath
    == f"data/utrecht_subset/AHN5_C_{int(bbox[0])}_{int(bbox[1])}_{subset[0]}_{subset[1]}.las"
][0]

lasfile_path

['data/utrecht_subset/AHN5_C_125000_456000_5_5.las', 'data/utrecht_subset/AHN5_C_125000_456000_8_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_9_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_2_1.las', 'data/utrecht_subset/AHN5_C_125000_456000_6_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_4_4.las', 'data/utrecht_subset/AHN5_C_125000_456000_3_9.las', 'data/utrecht_subset/AHN5_C_125000_456000_5_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_0_6.las', 'data/utrecht_subset/AHN5_C_125000_456000_1_9.las', 'data/utrecht_subset/AHN5_C_125000_456000_2_3.las', 'data/utrecht_subset/AHN5_C_125000_456000_7_0.las', 'data/utrecht_subset/AHN5_C_125000_456000_1_5.las', 'data/utrecht_subset/AHN5_C_125000_456000_3_5.las', 'data/utrecht_subset/AHN5_C_125000_456000_4_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_3_7.las', 'data/utrecht_subset/AHN5_C_125000_456000_5_7.las', 'data/utrecht_subset/AHN5_C_125000_456000_8_6.las', 'data/utrecht_subset/AHN5_C_125000_456000_2_6.las', 'data/utrec

'data/utrecht_subset/AHN5_C_125000_456000_5_5.las'

In [18]:
lasfile = lasdata_loader(lasfile_path, single_array=True)

De lasfile is nu geladen als een numpy array. Een matrix met de kolommen X,Y,Z en elke rij 1 punt.
je kan dit array ook filteren op index. Zie: https://numpy.org/doc/stable/user/basics.indexing.html
Python begint met tellen bij 0. Het derde punt in de lasfile krijg je daarom door:
```
lasfile[2, :]
```

Wil je alle Y-coordinaten? dan gebruik je:
```
lasfile[:, 1]
```

Beantwoord de volgende vragen:

In [37]:
# Vraag 1: hoeveel punten zitten er in deze lasfile? (antwoord: 1 regel code)
len(lasfile)

363668

In [38]:
# vraag 2: Hoeveel dimensies heeft deze lasfile? Deze vraag kan je beantwoorden zonder code.
lasfile.shape

(363668, 3)

In [39]:
# Vraag 3: wat is de x,y,z coordinaten van het 10e punt in deze lasfile? (antwoord: 1 regel code)
lasfile[9, :]

array([ 1.25502578e+05,  4.56597670e+05, -1.25000000e-01])

Daarnaast kan je ook dingen als min, max en gemiddelde berekenen van een subselectie. Probeer zelf erachter te komen hoe je dit doet! Beantwoord daarbij de volgende vragen:

In [40]:
# Vraag 4: Wat is de gemiddelde hoogte (z) van alle punten in deze lasfile? (antwoord: 1 regel code)
lasfile[:, 2].mean()

np.float64(0.24662079149114033)

In [41]:
# Vraag 5: Wat is de min en max X-waarde van alle punten in deze lasfile? (antwoord: 1 regel code)
lasfile[:, 0].min(), lasfile[:, 0].max()

(np.float64(125500.0), np.float64(125599.999))

In [42]:
# Vraag 6: Wat is de min en max Y-waarde van alle punten in deze lasfile? (antwoord: 1 regel code)
lasfile[:, 1].min(), lasfile[:, 1].max()

(np.float64(456500.0), np.float64(456599.999))

In [43]:
# Waarom had je deze X en Y waarden al kunnen verwachten zonder code?

Nu willen we de puntenwolk gaan "ophakken" in nog kleinere subsets. Vervolgens classificeren we alle punten die in een subset vallen ans behorend bij die subset.
Laten we de puntenwolk ophakken in een 3x3 matrix.

In [44]:
# Maak een kopie van de originele array en voeg een 4e kolom toe. Deze vullen we met 0-en.
lasfile_with_grid = np.zeros((len(lasfile), 4))
lasfile_with_grid[:, :3] = lasfile  # Kopieer X, Y, Z coordinaten

In [45]:
# Bereken de bounding box van de point cloud
x_min, x_max = lasfile_with_grid[:, 0].min(), lasfile_with_grid[:, 0].max()
y_min, y_max = lasfile_with_grid[:, 1].min(), lasfile_with_grid[:, 1].max()
print(f"X range: {x_min:.2f} to {x_max:.2f}")
print(f"Y range: {y_min:.2f} to {y_max:.2f}")

X range: 125500.00 to 125600.00
Y range: 456500.00 to 456600.00


In [46]:
# Bereken de cell grootte voor de 3x3 matrix
grid_size = 3
x_step = (x_max - x_min) / grid_size
y_step = (y_max - y_min) / grid_size

print(f"Cell grootte: {x_step:.2f} x {y_step:.2f}")

Cell grootte: 33.33 x 33.33


In [47]:
# Create a dictionary to store points for each grid cell
grid_cells = {}

# Per punt de grid cell assignment:
for i, point in enumerate(lasfile_with_grid):
    x, y, z = point[:3]

    # Bereken welk grid cell het punt in valt
    grid_x = int((x - x_min) / x_step)
    grid_y = int((y - y_min) / y_step)

    # Edge cases die precies op de grens vallen
    if grid_x == grid_size:
        grid_x = grid_size - 1
    if grid_y == grid_size:
        grid_y = grid_size - 1

    # Converteer grid coordinaten naar een enkele classificatie waarde (0-8)
    # Grid layout: 0 1 2
    #              3 4 5
    #              6 7 8
    grid_classification = grid_y * grid_size + grid_x

    # Voeg de grid classificatie toe als 4e dimensie
    lasfile_with_grid[i, 3] = grid_classification

    # Voeg het punt toe aan de dictionary
    if grid_classification not in grid_cells:
        grid_cells[grid_classification] = []
    grid_cells[grid_classification].append(point)

# Print statistics for each grid cell
print("Grid cell statistics:")
for cell_key in grid_cells.keys():
    if cell_key in grid_cells:
        point_count = len(grid_cells[cell_key])
        print(f"Cell {cell_key}: {point_count} points")
    else:
        print(f"Cell ({cell_key}): 0 points")

print(
    f"\nTotal points distributed: {sum(len(points) for points in grid_cells.values())}"
)
print(f"Original point count: {len(lasfile)}")

Grid cell statistics:
Cell 6: 27118 points
Cell 7: 38461 points
Cell 3: 28551 points
Cell 0: 39287 points
Cell 4: 47537 points
Cell 1: 51500 points
Cell 5: 42380 points
Cell 8: 47320 points
Cell 2: 41514 points

Total points distributed: 363668
Original point count: 363668


In [48]:
lasfile_with_grid[:4]

array([[ 1.25501602e+05,  4.56593789e+05, -3.90000000e-02,
         6.00000000e+00],
       [ 1.25501393e+05,  4.56594134e+05,  1.11000000e-01,
         6.00000000e+00],
       [ 1.25500738e+05,  4.56596006e+05, -7.90000000e-02,
         6.00000000e+00],
       [ 1.25501951e+05,  4.56594276e+05,  1.73800000e+00,
         6.00000000e+00]])

In [49]:
# Creëer een nieuwe numpy array met 4 dimensies: X, Y, Z, en grid classificatie
# De grid classificatie is een getal van 0-8 (voor een 3x3 grid)

# Maak een kopie van de originele array en voeg een 4e kolom toe
lasfile_with_grid = np.zeros((len(lasfile), 4))
lasfile_with_grid[:, :3] = lasfile  # Kopieer X, Y, Z coordinaten

# Voeg grid classificaties toe voor elk punt
for i, point in enumerate(lasfile):
    x, y, z = point

    # Bereken welke grid cell dit punt hoort
    grid_x = int((x - x_min) / x_step)
    grid_y = int((y - y_min) / y_step)

    # Handle edge case where point is exactly on the boundary
    if grid_x == grid_size:
        grid_x = grid_size - 1
    if grid_y == grid_size:
        grid_y = grid_size - 1

    # Converteer grid coordinaten naar een enkele classificatie waarde (0-8)
    # Grid layout: 0 1 2
    #              3 4 5
    #              6 7 8
    grid_classification = grid_y * grid_size + grid_x

    # Voeg de grid classificatie toe als 4e dimensie
    lasfile_with_grid[i, 3] = grid_classification

print("Nieuwe array shape:", lasfile_with_grid.shape)
print("Eerste 10 punten met grid classificatie:")
print("X\t\tY\t\tZ\t\tGrid")
for i in range(10):
    x, y, z, grid_class = lasfile_with_grid[i]
    print(f"{x:.2f}\t\t{y:.2f}\t\t{z:.2f}\t\t{int(grid_class)}")

# Toon hoeveel punten er in elke grid classificatie zitten
print("\nPunten per grid classificatie:")
unique_grids, counts = np.unique(lasfile_with_grid[:, 3], return_counts=True)
for grid_id, count in zip(unique_grids, counts):
    grid_x = int(grid_id) % grid_size
    grid_y = int(grid_id) // grid_size
    print(f"Grid {int(grid_id)} (cel {grid_x},{grid_y}): {count} punten")

Nieuwe array shape: (363668, 4)
Eerste 10 punten met grid classificatie:
X		Y		Z		Grid
125501.60		456593.79		-0.04		6
125501.39		456594.13		0.11		6
125500.74		456596.01		-0.08		6
125501.95		456594.28		1.74		6
125502.26		456595.29		-0.13		6
125501.05		456598.28		-0.07		6
125500.90		456598.65		-0.09		6
125501.66		456599.10		0.01		6
125501.38		456599.88		-0.03		6
125502.58		456597.67		-0.12		6

Punten per grid classificatie:
Grid 0 (cel 0,0): 39287 punten
Grid 1 (cel 1,0): 51500 punten
Grid 2 (cel 2,0): 41514 punten
Grid 3 (cel 0,1): 28551 punten
Grid 4 (cel 1,1): 47537 punten
Grid 5 (cel 2,1): 42380 punten
Grid 6 (cel 0,2): 27118 punten
Grid 7 (cel 1,2): 38461 punten
Grid 8 (cel 2,2): 47320 punten


Jammer genoeg biedt laspy geen mogelijkheid om pointclouds te visualiseren. Gelukkig kunnen we hier een ander package voor gebruiken die veel wordt gebruikt voor pointclouds en 3D modellen: open3D!

Naast visualiseren biedt dit package nog veel meer geavanceerde mogelijkheden voor 3D data in het algemeen. Lees meer op: https://www.open3d.org/docs/latest/introduction.html

In [50]:
# Genereer kleuren voor onze 9 grid cellen (0-8)
n_grid_cells = len(grid_cells)
grid_colors = generate_rgb_colors(n_grid_cells)

In [51]:
# Pas kleuren toe op onze punten
point_colors = apply_colors_to_points(lasfile_with_grid, grid_colors)


# Normaliseer kleuren naar 0-1 range voor Open3D
colors_normalized = point_colors.astype(np.float64) / 255.0

In [52]:
# Maak Open3D point cloud object
colored_geom = o3d.geometry.PointCloud()
colored_geom.points = o3d.utility.Vector3dVector(lasfile_with_grid[:, :3])
colored_geom.colors = o3d.utility.Vector3dVector(colors_normalized)

In [53]:
# Visualiseer
print("Visualisatie van de gekleurde point cloud wordt geopend...")
print("Elke kleur representeert een andere grid cel")
o3d.visualization.draw_geometries([colored_geom])

Visualisatie van de gekleurde point cloud wordt geopend...
Elke kleur representeert een andere grid cel


Als we de classificatie buiten python willen bekijken, kunnen we deze het beste opslaan als nieuwe las file. We moeten het numpy array weer terugveranderen in een las dataset zodat laspy deze kan opslaan. Door het numpy array om te zetten in lasdata zorgen we dat deze voldoet aan de specificaties voor lasdata (zoals een header met de juiste informatie).

In [46]:
las_classified = to_las(
    points=lasfile_with_grid[:, 0:3],
    colors=None,
    extra_dim={"grid_classification": lasfile_with_grid[:, 3].astype("uint32")},
)

In [59]:
las_path = output_folder / Path(Path(lasfile_path).stem + "_grid_classification.las")
las_classified.write(las_path)